In [2]:
import datetime
import pandas as pd
import geopandas as gpd

import FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from utils import timed

In [3]:
tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])
list_of_ts = list(FireTime.t_generator(tst, ted))

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [4]:
preprocess.preprocess_landcover()

2024-02-28 18:55:18,259 - FireLog - INFO - func:preprocess_landcover took: 23.96 sec


## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [5]:
preprocess.preprocess_region(region)

2024-02-28 18:58:37,262 - FireLog - INFO - func:preprocess_region took: 42.20 sec


## Once per input file

Next process each NRT file into half day files. First we'll get all the times that are of interest. This could also be done by inspecting all looking at all the files that exist and seeing which have not been preprocessed yet.

In [6]:
%%time
for sat in ["SNPP", "NOAA20"]:
    for t in list_of_ts[::2]:
        preprocess.preprocess_NRT_file(t, sat)

2024-02-28 18:58:37,268 - FireLog - INFO - preprocessing NRT file for 2023-8-28, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:238: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-02-28 18:58:39,096 - FireLog - INFO - func:preprocess_NRT_file took: 1.83 sec
2024-02-28 18:58:39,097 - FireLog - INFO - preprocessing NRT file for 2023-8-29, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:238: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-02-28 18:58:40,275 - FireLog - INFO - func:preprocess_NRT_file took: 1.18 sec
2024-02-28 18:58:40,276 - FireLog - INFO - preprocessing NRT file for 2023-8-30, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:238: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is depr

CPU times: user 14.4 s, sys: 545 ms, total: 14.9 s
Wall time: 24.5 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

Note: for the purpose of timing I am running the full month, but each of these steps could run in a separate proccess.

In [4]:
region = preprocess.read_region(region)

2024-02-28 19:00:50,879 - FireLog - INFO - func:read_region took: 195.86 ms


In [5]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)

2024-02-28 19:00:51,205 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, WesternUS
2024-02-28 19:00:51,238 - FireLog - INFO - func:read_preprocessed took: 31.93 ms
2024-02-28 19:00:51,266 - FireLog - INFO - func:read_preprocessed took: 27.58 ms
2024-02-28 19:00:51,270 - FireLog - INFO - func:preprocess_region_t took: 65.44 ms
2024-02-28 19:00:51,271 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, WesternUS
2024-02-28 19:00:51,316 - FireLog - INFO - func:read_preprocessed took: 44.30 ms
2024-02-28 19:00:51,361 - FireLog - INFO - func:read_preprocessed took: 44.18 ms
2024-02-28 19:00:51,366 - FireLog - INFO - func:preprocess_region_t took: 94.75 ms
2024-02-28 19:00:51,366 - FireLog - INFO - filtering and clustering 2023-8-29 AM, VIIRS, WesternUS
2024-02-28 19:00:51,395 - FireLog - INFO - func:read_preprocessed took: 28.58 ms
2024-02-28 19:00:51,423 - FireLog - INFO - func:read_preprocessed took: 26.52 ms
2024-02-28 19:01:32,442 - FireLog - INFO - func:do_

CPU times: user 3min 11s, sys: 530 ms, total: 3min 11s
Wall time: 3min 24s


## Run the fire expansion and merging algorithm

`Fire_Forward` is responsible for reading in the preprocessed data created in the `Ingest` notebook

In [4]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, restart=False, region=region)

2024-02-28 19:11:28,260 - FireLog - INFO - func:read_preprocessed took: 10.74 ms
2024-02-28 19:11:28,268 - FireLog - INFO - func:read_preprocessed took: 7.06 ms
2024-02-28 19:11:28,276 - FireLog - INFO - func:read_preprocessed took: 7.71 ms
2024-02-28 19:11:28,283 - FireLog - INFO - func:read_preprocessed took: 5.99 ms
2024-02-28 19:11:28,290 - FireLog - INFO - func:read_preprocessed took: 6.81 ms
2024-02-28 19:11:28,298 - FireLog - INFO - func:read_preprocessed took: 7.40 ms
2024-02-28 19:11:28,307 - FireLog - INFO - func:read_preprocessed took: 8.66 ms
2024-02-28 19:11:28,312 - FireLog - INFO - func:read_preprocessed took: 3.87 ms
2024-02-28 19:11:28,316 - FireLog - INFO - func:read_preprocessed took: 3.91 ms
2024-02-28 19:11:28,320 - FireLog - INFO - func:read_preprocessed took: 3.64 ms
2024-02-28 19:11:28,325 - FireLog - INFO - func:read_preprocessed took: 3.88 ms
2024-02-28 19:11:28,329 - FireLog - INFO - func:read_preprocessed took: 3.59 ms
2024-02-28 19:11:28,333 - FireLog - INF

We can save these files off in case something goes wrong later

In [5]:
postprocess.save_allpixels(allpixels, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, ted, region)

2024-02-28 19:13:27,669 - FireLog - INFO - func:save_allpixels took: 186.95 ms
2024-02-28 19:13:27,841 - FireLog - INFO - func:save_allfires_gdf took: 171.88 ms


## Write Snapshot files

Start by reading in the outputs from the run:

In [7]:
allpixels = postprocess.read_allpixels(ted, region)
allfires_gdf = postprocess.read_allfires_gdf(ted, region)

2024-02-28 19:22:07,335 - FireLog - INFO - func:read_allpixels took: 63.09 ms
2024-02-28 19:22:07,422 - FireLog - INFO - func:read_allfires_gdf took: 86.79 ms


Then iterate through all of the ts and save the snapshot layers.

In [9]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-02-28 19:22:21,425 - FireLog - INFO - func:save_snapshot_layers took: 415.79 ms
2024-02-28 19:22:21,847 - FireLog - INFO - func:save_snapshot_layers took: 418.61 ms
2024-02-28 19:22:22,314 - FireLog - INFO - func:save_snapshot_layers took: 462.75 ms
2024-02-28 19:22:22,816 - FireLog - INFO - func:save_snapshot_layers took: 498.63 ms
2024-02-28 19:22:23,372 - FireLog - INFO - func:save_snapshot_layers took: 551.80 ms
2024-02-28 19:22:23,947 - FireLog - INFO - func:save_snapshot_layers took: 570.85 ms
2024-02-28 19:22:24,556 - FireLog - INFO - func:save_snapshot_layers took: 605.07 ms
2024-02-28 19:22:25,193 - FireLog - INFO - func:save_snapshot_layers took: 633.35 ms
2024-02-28 19:22:25,813 - FireLog - INFO - func:save_snapshot_layers took: 616.10 ms
2024-02-28 19:22:26,458 - FireLog - INFO - func:save_snapshot_layers took: 641.12 ms
2024-02-28 19:22:27,105 - FireLog - INFO - func:save_snapshot_layers took: 642.86 ms
2024-02-28 19:22:27,849 - FireLog - INFO - func:save_snapshot_lay

CPU times: user 7.83 s, sys: 129 ms, total: 7.95 s
Wall time: 11.9 s


## Write Largefire files

First we will find all the large fires:

In [10]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-02-28 19:22:42,401 - FireLog - INFO - func:find_largefires took: 4.84 ms


First we'll use the `allpixels` object to create the `nplist` layer

In [11]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires)

2024-02-28 19:22:44,359 - FireLog - INFO - func:save_fire_nplist took: 55.39 ms
2024-02-28 19:22:44,400 - FireLog - INFO - func:save_fire_nplist took: 39.66 ms
2024-02-28 19:22:44,460 - FireLog - INFO - func:save_fire_nplist took: 58.99 ms
2024-02-28 19:22:44,514 - FireLog - INFO - func:save_fire_nplist took: 51.87 ms
2024-02-28 19:22:44,564 - FireLog - INFO - func:save_fire_nplist took: 48.66 ms
2024-02-28 19:22:44,610 - FireLog - INFO - func:save_fire_nplist took: 44.39 ms
2024-02-28 19:22:44,653 - FireLog - INFO - func:save_fire_nplist took: 42.27 ms
2024-02-28 19:22:44,704 - FireLog - INFO - func:save_fire_nplist took: 49.54 ms
2024-02-28 19:22:44,755 - FireLog - INFO - func:save_fire_nplist took: 49.01 ms
2024-02-28 19:22:44,800 - FireLog - INFO - func:save_fire_nplist took: 43.62 ms
2024-02-28 19:22:44,845 - FireLog - INFO - func:save_fire_nplist took: 43.55 ms
2024-02-28 19:22:44,894 - FireLog - INFO - func:save_fire_nplist took: 48.16 ms
2024-02-28 19:22:44,940 - FireLog - INFO

CPU times: user 928 ms, sys: 40.2 ms, total: 968 ms
Wall time: 2.05 s


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [12]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires)

2024-02-28 19:22:59,132 - FireLog - INFO - func:save_fire_layers took: 110.01 ms


53 rows that potentially need a merge


2024-02-28 19:22:59,257 - FireLog - INFO - func:save_fire_layers took: 123.48 ms
2024-02-28 19:22:59,363 - FireLog - INFO - func:save_fire_layers took: 105.17 ms
2024-02-28 19:22:59,387 - FireLog - INFO - func:merge_rows took: 22.99 ms
2024-02-28 19:22:59,495 - FireLog - INFO - func:save_fire_layers took: 107.07 ms
2024-02-28 19:22:59,511 - FireLog - INFO - func:merge_rows took: 14.80 ms
2024-02-28 19:22:59,614 - FireLog - INFO - func:save_fire_layers took: 102.90 ms
2024-02-28 19:22:59,718 - FireLog - INFO - func:save_fire_layers took: 102.66 ms
2024-02-28 19:22:59,829 - FireLog - INFO - func:save_fire_layers took: 110.35 ms
2024-02-28 19:22:59,937 - FireLog - INFO - func:save_fire_layers took: 106.84 ms
2024-02-28 19:23:00,041 - FireLog - INFO - func:save_fire_layers took: 102.75 ms
2024-02-28 19:23:00,197 - FireLog - INFO - func:save_fire_layers took: 154.83 ms
2024-02-28 19:23:00,306 - FireLog - INFO - func:save_fire_layers took: 107.06 ms
2024-02-28 19:23:00,412 - FireLog - INFO -

CPU times: user 1.25 s, sys: 68.5 ms, total: 1.32 s
Wall time: 3.79 s
